# E2E Testen met Vaadin TestBench

> tags: []

Learn how to do full-stack tests on a Flow application using Vaadin TestBench.

1.  [Documentatie](https://vaadin.com/docs/latest/)
2.  [Tutorial](https://vaadin.com/docs/latest/tutorial)
3.  End-to-end testen

-   [De basistestklasse maken](https://vaadin.com/docs/latest/tutorial/e2e-testing-with-testbench/#creating-the-base-test-class)
-   [De aanmeldingsweergave testen](https://vaadin.com/docs/latest/tutorial/e2e-testing-with-testbench/#testing-the-login-view)
-   [Een weergaveobject maken](https://vaadin.com/docs/latest/tutorial/e2e-testing-with-testbench/#creating-a-view-object)

End-to-end (e2e) tests worden gebruikt om de gehele applicatie te testen. Ze zijn veel grover dan eenheids- of integratietests. Dit maakt ze zeer geschikt om te controleren of de applicatie als geheel werkt en om eventuele regressies op te vangen die door meer specifieke tests kunnen worden gemist.

End-to-end tests worden uitgevoerd in een browservenster, bestuurd door een webstuurprogramma en uitgevoerd op de server waar de toepassing wordt geïmplementeerd. Vaadin TestBench zorgt voor dit alles.

<table><tbody><tr><td _msthidden="1"><p>Note</p></td><td><p>Vaadin TestBench is een commercieel product</p><p _msthash="2265393" _msttexthash="46340996">De end-to-end tests maken gebruik van <a href="https://vaadin.com/testbench" _istranslated="1">Vaadin TestBench</a>, een commerciële tool die deel uitmaakt van het Vaadin Pro Subscription. U kunt een gratis proefversie krijgen bij <a href="https://vaadin.com/trial?utm_source=github+vaadin" _istranslated="1">https://vaadin.com/trial</a>. Alle Vaadin Pro tools en componenten zijn gratis voor studenten via het <a href="https://education.github.com/pack" _istranslated="1">GitHub Student Developer Pack</a>.</p></td></tr></tbody></table>

## [](https://vaadin.com/docs/latest/tutorial/e2e-testing-with-testbench/#creating-the-base-test-class)De basistestklasse maken

Om herhaling in elke testklasse te voorkomen, is het een goed idee om gemeenschappelijke logica in een abstracte klasse te plaatsen en alle tests deze klasse te laten uitbreiden. Het grootste deel van het zware werk met betrekking tot het starten van browsers, enz. wordt afgehandeld in TestBench, maar er zijn een paar nuttige dingen die u aan de abstracte klasse kunt toevoegen.`ParallelTest`

Maak een nieuwe klasse in het pakket. Zorg ervoor dat u het in en niet .`AbstractTest``com.example.application.it``src/test/java``src/main/java`

```
package com.example.application.it;

import ch.qos.logback.classic.Level;
import ch.qos.logback.classic.Logger;
import com.vaadin.testbench.IPAddress;
import com.vaadin.testbench.ScreenshotOnFailureRule;
import com.vaadin.testbench.parallel.ParallelTest;
import io.github.bonigarcia.wdm.WebDriverManager;
import org.junit.Before;
import org.junit.BeforeClass;
import org.junit.Rule;
import org.slf4j.LoggerFactory;

public abstract class AbstractTest extends ParallelTest {
    private static final String SERVER_HOST = IPAddress.findSiteLocalAddress();
    private static final int SERVER_PORT = 8080;
    private final String route;

    static {
        // Prevent debug logging from Apache HTTP client
        Logger root = (Logger) LoggerFactory.getLogger(Logger.ROOT_LOGGER_NAME);
        root.setLevel(Level.INFO);
    }

    @BeforeClass
    public static void setupClass() {
        WebDriverManager.chromedriver().setup(); 1
    }

    @Rule 2
    public ScreenshotOnFailureRule rule = new ScreenshotOnFailureRule(this, true);

    @Before
    public void setup() throws Exception {
        super.setup();
        getDriver().get(getURL(route)); 3
    }

    protected AbstractTest(String route) {
        this.route = route;
    }

    private static String getURL(String route) {
        return String.format("http://%s:%d/%s", SERVER_HOST, SERVER_PORT, route);
    }
}
```

1.  Start by invoking the **Chrome** before any test method is invoked. TestBench does not invoke the web driver manager.`WebDriverManager`
    
2.  `ScreenshotOnFailureRule` tells TestBench to grab a screenshot before exiting, if a test fails. This helps you understand what went wrong when tests do not pass.
    
3.  Open the browser to the correct URL before each test. For this, you need the host name where the application runs ("localhost" in development), the port the server uses (set to 8080 in application.properties), and information about the route to start from.
    

## [](https://vaadin.com/docs/latest/tutorial/e2e-testing-with-testbench/#testing-the-login-view)Testing the Login View

Now that your setup is complete, you can start developing your first test: ensuring that a user can log in. For this test, you need to open the base URL.

Create a new class, , in the same package as . The test validates that logging in with the correct user and password succeeds.`LoginIT``AbstractTest`

```
package com.example.application.it;

import com.vaadin.flow.component.login.testbench.LoginFormElement;
import org.junit.Assert;
import org.junit.Test;

public class LoginIT extends AbstractTest {
    public LoginIT() {
        super("");
    }

    @Test
    public void loginAsValidUserSucceeds() {
        // Find the LoginForm used on the page
        LoginFormElement form = $(LoginFormElement.class).first();
        // Enter the credentials and log in
        form.getUsernameField().setValue("user");
        form.getPasswordField().setValue("userpass");
        form.getSubmitButton().click();
        // Ensure the login form is no longer visible
        Assert.assertFalse($(LoginFormElement.class).exists());
    }
}
```

<table><tbody><tr><td _msthidden="1"><p>Note</p></td><td><p>Make sure tests have the correct name</p><p><span _mstmutation="1" _msthash="2705777" _msttexthash="27281813">The name of the class should end in ”IT” for the test runner to pick it up as an integration test. If you name it , it will be run as a unit test. Hence, the server will not be started and the test will fail.</span><code>LoginTest</code></p></td></tr></tbody></table>

<table><tbody><tr><td _msthidden="1"><p>Tip</p></td><td><p>Start the server separately to speed up tests</p><p><span _mstmutation="1" _msthash="2705778" _msttexthash="33288190">While developing tests, it is not very efficient to run the tests as . Instead, you can start the server manually by launching the class or with . You can then execute the selected test in your IDE and, hence, do not have to wait for the server to start every time.</span><code>mvn -Pit verify</code><code>Application</code><code>spring-boot:run</code></p></td></tr></tbody></table>

Start the application normally, then right-click and select **Run 'LoginIT'**.`LoginIT.java`

The first time you run the test, you will be asked to start a trial or to validate your existing license. Follow the instructions in the browser window that opens.

## [](https://vaadin.com/docs/latest/tutorial/e2e-testing-with-testbench/#creating-a-view-object)Creating a View Object

You can now add a second test: validating that you cannot log in with an invalid password.

For this test, you need to write the same code to access the components in the view as you did for the first test. To make your tests more maintainable, you can create a view object (otherwise known as a call page object or element class) for each view. A view object provides a high-level API to interact with the view and hides the implementation details.

For the login view, create the class in a new package, :`LoginViewElement``com.example.application.it.elements`

```
package com.example.application.it.elements;

import com.vaadin.flow.component.login.testbench.LoginFormElement;
import com.vaadin.flow.component.orderedlayout.testbench.VerticalLayoutElement;
import com.vaadin.testbench.annotations.Attribute;

@Attribute(name = "class", contains = "login-view")
public class LoginViewElement extends VerticalLayoutElement {

    public boolean login(String username, String password) {
        LoginFormElement form = $(LoginFormElement.class).first();
        form.getUsernameField().setValue(username);
        form.getPasswordField().setValue(password);
        form.getSubmitButton().click();

        // Return true if we end up on another page
        return !$(LoginViewElement.class).onPage().exists();
    }

}
```

<table><tbody><tr><td _msthidden="1"><p>Caution</p></td><td><p>Class hierarchies must match</p><p><span _mstmutation="1" _msthash="2705779" _msttexthash="10625017">To make the correct functionality available from superclasses, the hierarchy of the view object should match the hierarchy of the view ( vs ).</span><code>public class LoginView extends VerticalLayout</code><code>public class LoginViewElement extends VerticalLayoutElement</code></p></td></tr></tbody></table>

Adding the annotation allows you to find the using the TestBench query API, for example:`@Attribute(name = "class", contains = "login-view")``LoginViewElement`

```
LoginViewElement loginView = $(LoginViewElement.class).onPage().first();
```

The annotation searches for the class name, which is set for the login view in the constructor. The call ensures that the whole page is searched. By default, a query starts from the active element.`login-view``onPage()``$`

Now that you have the class, you can simplify your test to be:`LoginViewElement``loginAsValidUserSucceeds()`

```
@Test
public void loginAsValidUserSucceeds() {
    LoginViewElement loginView = $(LoginViewElement.class).onPage().first();
    Assert.assertTrue(loginView.login("user", "userpass"));
}
```

Add a test to use an invalid password as follows:

```
@Test
public void loginAsInvalidUserFails() {
    LoginViewElement loginView = $(LoginViewElement.class).onPage().first();
    Assert.assertFalse(loginView.login("user", "invalid"));
}
```

Continue testing the other views by creating similar view objects and IT classes.

The next chapter covers how to make a production build of the application and deploy it to a cloud platform.

0DDF0F9E-DCF0-4AEC-9DD4-C241699CC7F7

[TutorialUnit and Integration Testing](https://vaadin.com/docs/latest/tutorial/unit-and-integration-testing)[TutorialProduction Deployment](https://vaadin.com/docs/latest/tutorial/production-deployment)

**Was this page helpful?**  
Leave a comment or a question below. You can also join the [chat on Discord](https://discord.gg/MYFq5RTbBn) or [ask questions on StackOverflow](https://stackoverflow.com/questions/tagged/vaadin).
